# ITS8080 Project — Tasks 3-11 Report

This notebook documents the data analysis and modelling tasks.

## Task 3: Data Visualisation

We visualize demand, PV, and price to understand the dataset.

### Key Java Implementation (TimeSeriesPlotter.java)

```java
public void plotDemandPvPriceFirstDays(List<DataRow> rows) {
    var window = firstDays(rows, cfg.days());
    var points = align(window);
    var chart = chart("Prices, Demand and PV");
    chart.addSeries("Demand (kW)", points.time(), points.demand());
    chart.addSeries("PV (kW)", points.time(), points.pv());
    chart.addSeries("Price", points.timePrice(), points.price()).setYAxisGroup(1);
    ChartExporter.savePng(chart, out, "timeseries_demand_pv_price");
}
```

### Results
Figures are saved in `figures/`.

In [ ]:
from IPython.display import Image, display
print("Task 3: Time Series Plot")
display(Image(filename='../figures/timeseries_demand_pv_price.png'))

print("Task 3: Price Histogram")
display(Image(filename='../figures/histogram_price.png'))

print("Task 3: Demand vs Temperature Scatter")
display(Image(filename='../figures/scatter_demand_vs_temperature.png'))

## Task 4: Missing Data Analysis

We analyze and impute missing PV data.

### Key Java Implementation (PvMod1Imputation.java)

```java
public static ImputationResult runAll(List<DataRow> rows, PvTask4Config cfg) {
    // ...
    var uni = UnivariateImputation.interpolate(original);
    var multi = MultivariateImputation.impute(original, weather, cfg);
    return new ImputationResult(original, uni, multi);
}
```

In [ ]:
print("Task 4: PV Imputation Results")
display(Image(filename='../figures/pv_mod1_before_after.png'))

## Task 5: Feature Engineering

Normality checks and feature ranking.

### Key Java Implementation (FeatureRanking.java)

```java
public static List<FeatureScore> rankByAbsPearson(Dataset ds) {
    // ...
    var corr = new PearsonsCorrelation();
    double r = corr.correlation(featVec, targetVec);
    scores.add(new FeatureScore(name, Math.abs(r)));
    // ...
    return scores;
}
```

In [ ]:
print("Task 5: Demand Histogram")
display(Image(filename='../figures/task5_histogram_demand.png'))

print("Task 5: Demand vs Temperature Scatter")
display(Image(filename='../figures/task5_scatter_demand_vs_temperature.png'))

## Task 6: Decomposition

Classical additive decomposition.

### Key Java Implementation (ClassicalDecomposition.java)

```java
// Trend: Moving Average
double[] trend = MovingAverage.centered(observed, window);

// Seasonality: Average detrended by hour
double[] seasonal = new double[n];
for (int i=0; i<n; i++) seasonal[i] = seasonalIndices[i % period];

// Residual
double[] residual = new double[n];
for (int i=0; i<n; i++) residual[i] = observed[i] - trend[i] - seasonal[i];
```

In [ ]:
print("Task 6: Typical Profiles")
display(Image(filename='../figures/task6_typical_profiles.png'))

## Task 7: Statistical Modelling (AR)

Autoregressive models on differenced data.

### Key Java Implementation (ArDemandModel.java)

```java
public double[] forecastLevels(double[] yTrain, int steps) {
    var dy = Differencing.diff1(yTrain);
    var phi = fitArOnDiff(dy, order);
    // Forecast diffs and integrate back
    // ...
}
```

In [ ]:
print("Task 7: ACF Plot")
display(Image(filename='../figures/task7_acf.png'))

print("Task 7: PACF Plot")
display(Image(filename='../figures/task7_pacf.png'))

## Task 8: Machine Learning (XGBoost)

Training an XGBoost model with lagged features.

### Key Java Implementation (TrainXgb.java)

```java
public static Booster fit(DemandSupervisedDataset train, DemandSupervisedDataset valid, Task8Config cfg) {
    var params = new HashMap<String, Object>();
    params.put("max_depth", cfg.maxDepth());
    params.put("eta", cfg.eta());
    params.put("objective", "reg:squarederror");
    
    var dtrain = new DMatrix(train.X(), train.y());
    var dvalid = new DMatrix(valid.X(), valid.y());
    
    return XGBoost.train(dtrain, params, cfg.numRounds(), watches, ...);
}
```

## Task 9: Rolling-Origin Forecasting

Evaluating models (AR, XGBoost, Naive) on out-of-sample data.

### Key Java Implementation (RollingDayByDay.java)

```java
public static ForecastReport run(List<Double> history, List<ForecastExoRow> horizon, List<ModelSpec> models, Task9Config cfg) {
    // For each day in horizon
    for (int d = 0; d < days; d++) {
        // Train models on history
        // Forecast next 24 hours
        // Update history with actuals
    }
    return ForecastReport.from(dayResults);
}
```

## Task 10: Models with Exogenous Inputs

Adding weather and time features to AR (ARX) and XGBoost.

### Key Java Implementation (XgbFeaturesExo.java)

```java
public static float[] makeFeatures(PastRow[] lags, ForecastExoRow current, boolean useExo) {
    // Add lag features
    // ...
    if (useExo) {
        x[c++] = (float) current.timestamp().getHour();
        x[c++] = (float) current.temperature();
        x[c++] = (float) current.cloudCover();
        // ...
    }
    return x;
}
```

## Task 11: Optimal Control of Storage

Minimizing cost using battery storage.

### Key Java Implementation (Task11BatteryOptimizer.java)

```java
// Core optimization loop
for (int t = 0; t < n; t++) {
    // Variables
    charge[t] = model.addVariable("ch_" + t).lower(ZERO).upper(bd(battery.chargeLimitKW()));
    gridIn[t] = model.addVariable("gridIn_" + t).lower(ZERO).upper(bd(config.gridLimitKW()));
    
    // Balance Constraint
    model.addExpression("balance_" + t)
        .level(bd(demand))
        .set(gridIn[t], ONE).set(pvUsed[t], ONE).set(discharge[t], ONE)
        .set(charge[t], ONE.negate()).set(gridOut[t], ONE.negate());

    // Objective
    obj.set(gridIn[t], bd(price));
    obj.set(gridOut[t], bd(-config.exportPriceFactor() * price));
}
```

### Results Analysis (Load CSV)

In [ ]:
import pandas as pd

try:
    df_low = pd.read_csv("../figures/task11_schedule_pv_low.csv")
    print("--- PV LOW SCENARIO ---")
    print(df_low.head())
except Exception as e:
    print(f"Could not load PV Low file: {e}")

try:
    df_high = pd.read_csv("../figures/task11_schedule_pv_high.csv")
    print("\n--- PV HIGH SCENARIO ---")
    print(df_high.head())
except Exception as e:
    print(f"Could not load PV High file: {e}")